### Download Grammar

In [14]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [15]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [16]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

### Node Types

In [17]:
print(bert_tokenizer.node_types)

['*', '=', 'finally', 'decorator', '>>=', 'assert_statement', 'print_statement', 'block', '**', ')', 'tuple', 'list_comprehension', 'case_pattern', 'raise_statement', 'list_splat_pattern', 'class', '%', '|=', '_simple_statement', 'else', '_compound_statement', ']', 'binary_operator', 'future_import_statement', 'print', 'integer', ':', '__future__', 'from', 'identifier', 'dotted_name', '[', '~', 'continue', 'except_clause', 'break_statement', 'elif', 'dictionary', 'type', 'module', '>', 'parenthesized_expression', 'with', '(', 'delete_statement', 'keyword_separator', 'with_statement', 'relative_import', 'try', 'pair', 'global_statement', 'dictionary_comprehension', '<<=', '<', 'parameter', '!=', 'while_statement', 'parenthesized_list_splat', 'comparison_operator', 'break', 'exec_statement', 'elif_clause', '+', 'del', 'with_clause', '+=', 'raise', 'chevron', '^=', '@=', 'match', 'ellipsis', '*=', 'match_statement', 'await', 'assert', 'comment', '<<', 'not', '&', 'positional_separator', '

### Encodings

In [18]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
#code = "def hello_world(a,b):\n    print('hello world')"
#code = "def __ordered_values_by_indexes(self, data, inds): \"\"\" Return values (intensities) by indexes. Used for multiscale graph cut. data = [[0 1 1], [0 2 2], [0 2 2]] inds = [[0 1 2], [3 4 4], [5 4 4]] return: [0, 1, 1, 0, 2, 0] If the data are not consistent, it will take the maximal value \"\"\" # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
#code = "def __ordered_values_by_indexes(self, data, inds):  # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
target_node_type = "*"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type))

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def multiply_numbers(a,b):
    return a<mask>b


### Code Prediction

In [19]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

### Evaluation

In [20]:
import CodeCheckList.utils as utils

prediction_number = 0
print('------------- CODE -------------')
print(code)
print('\n---------- MASKED -------------')
print(masked_code)
print('\n--------- PREDICTED -----------')
predicted_code = predictions[prediction_number]
print(predicted_code)
print('\n--------- AST COMPARE -----------')
filtered_nodes = []
filtered_nodes_predict = []
utils.find_nodes(bert_tokenizer.parser.parse(bytes(code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes)
utils.find_nodes(bert_tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes_predict)
print(len(filtered_nodes))
print(len(filtered_nodes_predict))
#base the evaluation on size comparison

------------- CODE -------------
def multiply_numbers(a,b):
    return a*b

---------- MASKED -------------
def multiply_numbers(a,b):
    return a<mask>b

--------- PREDICTED -----------
def multiply_numbers(a,b):
    return a,b

--------- AST COMPARE -----------
1
0


In [21]:
from CodeCheckList.evaluator import Evaluator

evaluator = Evaluator(checkpoint, python_language)

results = evaluator.evaluate_code_snippet(code, 5)

print(results)

[{'*': [[1, 0, False], [1, 1, True], [1, 0, False], [1, 0, False], [1, 0, False]]}, {'=': []}, {'finally': []}, {'decorator': []}, {'>>=': []}, {'assert_statement': []}, {'print_statement': []}, {'block': [[1, 1, True], [1, 1, True], [1, 1, True], [1, 1, True], [1, 1, True]]}, {'**': []}, {')': [[1, 1, True], [1, 1, True], [1, 1, True], [1, 1, True], [1, 1, True]]}, {'tuple': []}, {'list_comprehension': []}, {'case_pattern': []}, {'raise_statement': []}, {'list_splat_pattern': []}, {'class': []}, {'%': []}, {'|=': []}, {'_simple_statement': []}, {'else': []}, {'_compound_statement': []}, {']': []}, {'binary_operator': [[1, 0, False], [1, 0, False], [1, 1, True], [1, 0, False], [1, 0, False]]}, {'future_import_statement': []}, {'print': []}, {'integer': []}, {':': [[1, 1, True], [1, 1, True], [1, 1, True], [1, 1, True], [1, 1, True]]}, {'__future__': []}, {'from': []}, {'identifier': [[5, 5, True], [5, 5, True], [5, 6, True], [5, 5, True], [5, 6, True]]}, {'dotted_name': []}, {'[': []},

### Code Search Net

In [22]:
from datasets import load_dataset 
import CodeCheckList.utils as utils

max_token_number = bert_tokenizer.tokenizer.max_len_single_sentence
print(max_token_number)

test_set = load_dataset("code_search_net", split='test')
test_set = test_set.filter(lambda sample: True if sample['language']== python_language
            and len(sample['func_code_tokens']) <= max_token_number
            and len(bert_tokenizer.tokenizer(sample['whole_func_string'])['input_ids']) <= max_token_number else False, num_proc=1)

test_set = utils.get_random_sub_set_test_set(utils.get_test_sets(load_dataset("code_search_net", split='test'), "python", evaluator.tokenizer.tokenizer.max_len_single_sentence, evaluator.tokenizer), 2)

len(test_set)

510


No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
100%|██████████| 101/101 [00:39<00:00,  2.54ba/s]
No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
100%|██████████| 101/101 [00:39<00:00,  2.53ba/s]


2

In [23]:
number_of_predictions = 2
save_path = "example"

print("evaluate")
results_dataframe = evaluator(test_set, number_of_predictions)
print(results_dataframe)

evaluate
              ast_element  occurences successful_predictions  \
0                       *           1                 [1, 1]   
1                       =           5                 [1, 0]   
2                 finally           0                 [0, 0]   
3               decorator           0                 [0, 0]   
4                     >>=           0                 [0, 0]   
..                    ...         ...                    ...   
190  augmented_assignment           3                 [0, 0]   
191                   //=           0                 [0, 0]   
192        unary_operator           1                 [1, 0]   
193    continue_statement           0                 [0, 0]   
194                    {{           0                 [0, 0]   

    failed_predictions total_predictions success_average failure_average  
0               [0, 0]            [1, 1]      [1.0, 1.0]      [0.0, 0.0]  
1               [0, 1]            [1, 1]      [1.0, 0.0]      [0.0, 1.0]

### Module

In [24]:
#| default_exp evaluator

In [25]:
#| export
import CodeCheckList
import pandas as pd

import CodeCheckList.utils as utils
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker
from CodeCheckList.predictor import Predictor


In [26]:
#| export
class Evaluator:
    """Evaluator Module to perform all AST Evaluations"""
    def __init__(self, checkpoint: str, language):
        self.tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)
        self.masker = Masker(self.tokenizer)
        self.predictor = Predictor.from_pretrained(checkpoint, self.tokenizer)

    def __call__(self, test_set, number_of_predictions: int):
        results_dict = self.evaluate_test_set(test_set, number_of_predictions)
        results_dataframe = pd.DataFrame([], columns=['ast_element', 'occurences', 'successful_predictions', 'failed_predictions', 'total_predictions', 'success_average', 'failure_average'])
        for result_index, result in enumerate(results_dict):
            results_dataframe.loc[len(results_dataframe.index)] = [self.tokenizer.node_types[result_index] ,result[0], result[1], result[2], result[3], result[4], result[5]]
        return results_dataframe
    
    def evaluate_test_set(self, test_set, number_of_predictions: int):
        results_dict = []
        for node_type in self.tokenizer.node_types:
            results_dict.append([0, [0 for i in range(0,number_of_predictions)], 
                                [0 for i in range(0,number_of_predictions)], 
                                [0 for i in range(0,number_of_predictions)], 
                                [0 for i in range(0,number_of_predictions)],
                                [0 for i in range(0,number_of_predictions)]])
        for sample in test_set:
            sample_results = self.evaluate_code_snippet(sample['whole_func_string'], number_of_predictions)
            for element_index, element_result in enumerate(sample_results):
                element_result_values = list(element_result.values())[0]
                if len(element_result_values) > 0:
                    results_dict[element_index][0] += element_result_values[0][0]
                    for prediction_number_index in range(0, number_of_predictions):
                        results_dict[element_index][1][prediction_number_index]+= (1 if element_result_values[prediction_number_index][2] else 0)
                        results_dict[element_index][2][prediction_number_index]+= (0 if element_result_values[prediction_number_index][2] else 1)
                        results_dict[element_index][3][prediction_number_index]= results_dict[element_index][1][prediction_number_index] + results_dict[element_index][2][prediction_number_index]
                        results_dict[element_index][4][prediction_number_index]= results_dict[element_index][1][prediction_number_index]/results_dict[element_index][3][prediction_number_index]
                        results_dict[element_index][5][prediction_number_index]= results_dict[element_index][2][prediction_number_index]/results_dict[element_index][3][prediction_number_index]
        return results_dict
        
    def evaluate_code_snippet(self, source_code: str, number_of_predictions: int):
        evaluation_results = []
        for node_type_idx, node_type in enumerate(self.tokenizer.node_types):
            evaluation_results.append({node_type: self.evaluate_node_type_on_snippet(source_code, node_type_idx, number_of_predictions)})
        return evaluation_results
            
    def evaluate_node_type_on_snippet(self, source_code: str, target_node_type_idx: int, number_of_predictions: int):
        results=[]
        source_code_nodes = []
        utils.find_nodes(self.tokenizer.parser.parse(bytes(source_code, "utf8")).root_node, 
            self.tokenizer.node_types[target_node_type_idx], source_code_nodes)
        if len(source_code_nodes) == 0:
            return results

        #source_code_encoding = self.tokenizer(source_code)
        masked_code_encoding = self.masker(source_code, self.tokenizer(source_code), target_node_type_idx)
        #masked_code = self.tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == self.tokenizer.tokenizer.bos_token_id or 
        #    token_id == self.tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))
        predictions = self.predictor(masked_code_encoding, self.tokenizer.tokenizer(source_code, return_tensors='pt'), number_of_predictions)       

        for prediction_number in range(0, number_of_predictions):
            predicted_code = predictions[prediction_number]
            predicted_nodes = []
            utils.find_nodes(self.tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, self.tokenizer.node_types[target_node_type_idx], predicted_nodes)
            results.append([len(source_code_nodes), len(predicted_nodes), len(predicted_nodes)>=len(source_code_nodes)])

        return results